In [33]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [34]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [35]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [36]:
from pymongo import MongoClient
import pymongo

In [37]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2016 = db.rm_patent_cn_2016
    db_rm_us_2016 = db.rm_patent_us_2016
    db_rm_de_2016 = db.rm_patent_de_2016

    
except errors.ConnectionFailure as err:
    print(err)

# 2016

## Company數量

In [38]:
data_assignee = db_rm_de_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [39]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [40]:
data_assignee = db_rm_cn_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [41]:
data_assignee = db_rm_us_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [42]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [43]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
1160,Samsung Electronics Co Ltd,775
706,Intel Corp,292
1075,Qualcomm Inc,196
1452,Telefonaktiebolaget LM Ericsson AB,166
632,Huawei Technologies Co Ltd,132
1655,ZTE Corp,129
803,LG Electronics Inc,92
716,International Business Machines Corp,91
285,Cisco Technology Inc,88
69,Anhui Sharetronic Iot Technology Co Ltd,85


In [44]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [45]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Samsung Electronics Co Ltd,775,6.652863
1,Intel Corp,292,5.676754
2,Qualcomm Inc,196,5.278115
3,Telefonaktiebolaget LM Ericsson AB,166,5.111988
4,Huawei Technologies Co Ltd,132,4.882802
5,ZTE Corp,129,4.859812
6,LG Electronics Inc,92,4.521789
7,International Business Machines Corp,91,4.510860
8,Cisco Technology Inc,88,4.477337
9,Anhui Sharetronic Iot Technology Co Ltd,85,4.442651


公司名稱正規化

In [46]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba"))

In [47]:
com_df

,company,count,count_log
0,Samsung Electronics,775,6.652863
1,Intel,292,5.676754
2,Qualcomm,196,5.278115
3,Telefonaktiebolaget LM Ericsson AB,166,5.111988
4,Huawei Technologies,132,4.882802
5,ZTE,129,4.859812
6,LG Electronics,92,4.521789
7,IBM,91,4.510860
8,Cisco Technology,88,4.477337
9,Anhui Sharetronic Iot Technology,85,4.442651


In [48]:
com_ls = list(com_df.company)

# Data 2016

富比士2000資料

In [49]:
f_data = pd.read_csv("./Forbes/Forbes_2016.csv")

In [50]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue
18,Samsung Electronics,161.550,177.286,16.5176,206.541,18,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Oh-Hyun Kwon,http://www.forbes.com/companies/samsung-electr...,0.079972,0.093169
59,Intel,149.259,56.276,11.4740,105.467,60,Information Technology,Semiconductors,North America,United States,NaN,NaN,Brian Krzanich,http://www.forbes.com/companies/intel/,0.108792,0.203888
136,Taiwan Semiconductor,125.605,25.661,9.0939,50.462,137,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Te Liu,http://www.forbes.com/companies/taiwan-semicon...,0.180213,0.354386
168,Qualcomm,77.324,22.614,4.9070,50.060,169,Information Technology,Semiconductors,North America,United States,California,California,Steven Mollenkopf,http://www.forbes.com/companies/qualcomm/,0.098022,0.216989
377,SK Hynix,17.513,16.609,3.8190,25.311,377,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Seong-Wook Park,http://www.forbes.com/companies/sk-hynix/,0.150883,0.229936
393,Texas Instruments,58.989,13.000,2.9440,16.230,394,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard Templeton,http://www.forbes.com/companies/texas-instrume...,0.181392,0.226462
559,NXP Semiconductors,29.199,6.101,1.5260,26.354,560,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Richard Clemmer,http://www.forbes.com/companies/nxp-semiconduc...,0.057904,0.250123
578,ASML Holding,41.325,6.972,1.7959,15.767,579,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter Wennink,http://www.forbes.com/companies/asml-holding/,0.113902,0.257587
582,Micron Technology,11.055,13.737,1.0710,24.819,583,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,D Durcan,http://www.forbes.com/companies/micron-technol...,0.043152,0.077965
633,Applied Materials,23.858,9.557,1.3150,13.326,634,Information Technology,Semiconductors,North America,United States,California,California,Gary Dickerson,http://www.forbes.com/companies/applied-materi...,0.098679,0.137595


In [51]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [52]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [53]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
0,Samsung Electronics,775,6.652863
1,Intel,292,5.676754
2,Qualcomm,196,5.278115


In [54]:
# pbc 0.09, 0.09
comp_ls = ["Qualcomm","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB","ZTE"]

In [55]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [56]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
1,Intel,292,5.676754
2,Qualcomm,196,5.278115
3,Telefonaktiebolaget LM Ericsson AB,166,5.111988
4,Huawei Technologies,132,4.882802
5,ZTE,129,4.859812


In [57]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.18812983439318598
ari:  0.3477508650519031


## 使用Forbes做join

In [58]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [59]:
# pbc 0.09, 0.09
comp_ls = ["Qualcomm","Intel","Huawei Technologies","Telefonaktiebolaget LM Ericsson AB","ZTE"]

In [60]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [61]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.08516709614785488
ari:  0.13388585214954302
